**Загрузка датасета и импорт библиотек**

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

od.download('https://www.kaggle.com/datasets/martaarroyo/palmer-penguins-for-binary-classification/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: daniillosev
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/martaarroyo/palmer-penguins-for-binary-classification


100%|██████████| 2.42k/2.42k [00:00<00:00, 2.22MB/s]

In [3]:
df = pd.read_csv('/content/palmer-penguins-for-binary-classification/penguins_binary_classification.csv')

**Анализ данных**

In [ ]:
df

species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
3    Adelie  Torgersen            36.7           19.3              193.0   
4    Adelie  Torgersen            39.3           20.6              190.0   
..      ...        ...             ...            ...                ...   
269  Gentoo     Biscoe            47.2           13.7              214.0   
270  Gentoo     Biscoe            46.8           14.3              215.0   
271  Gentoo     Biscoe            50.4           15.7              222.0   
272  Gentoo     Biscoe            45.2           14.8              212.0   
273  Gentoo     Biscoe            49.9           16.1              213.0   

     body_mass_g  year  
0         3750.0  2007  
1         3800.0  2007  
2         3250.0  2007  
3         3450.0  2007  
4         3650.0  2007  
..           ...   ...  
269       4925.0  2009  
270       4850.0  2009  
271       5750.0  2009  
272       5200.0  2009  
273       5400.0  2009  

[274 rows x 7 columns]

Данный датасет содержит информацию о  различных видах пингвинов и состоит из:


*   Вид пингвинов
*   Остров обитания
*   Длина клюва
*   Высота клюва
*   Длина плавника
*   Масса пингвина
*   Год обнаружения?



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            274 non-null    object 
 1   island             274 non-null    object 
 2   bill_length_mm     274 non-null    float64
 3   bill_depth_mm      274 non-null    float64
 4   flipper_length_mm  274 non-null    float64
 5   body_mass_g        274 non-null    float64
 6   year               274 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 15.1+ KB


**Вывод:** два признака выражены текстовой переменной, необходимо перевести их в числа

In [ ]:
df.isna().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
year                 0
dtype: int64

**Вывод:** заполнение пропусков не нужно

In [ ]:
df['species'].value_counts()

species
Adelie    151
Gentoo    123
Name: count, dtype: int64

In [ ]:
df['island'].value_counts()

island
Biscoe       167
Dream         56
Torgersen     51
Name: count, dtype: int64

**Вывод:** в датасете хранится информация только о 2 видах пингвинов и о 3 островах

**Предобработка данных**

Для перевода строковых признаков в числовые будет использован кодировщик OneHotEncoder, так как различных значений в датсете мало и, в отличие от LabelEncoder, будущий алгоритм не будет воспринимать закодированные категории одну приоритетнее другой. Для целевой переменной применим LabelEncoder, так как пространство должно быть одномерное

In [ ]:
x = df.drop(['species'], axis = 1)
y = df['species']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x_train = x_train.reset_index()
x_train = x_train.drop(['index'], axis = 1)
y_train = y_train.reset_index()
y_train = y_train.drop(['index'], axis = 1)
x_test = x_test.reset_index()
x_test = x_test.drop(['index'], axis = 1)
y_test = y_test.reset_index()
y_test = y_test.drop(['index'], axis = 1)

In [ ]:
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(x_train['island'].values.reshape(-1, 1))

OneHotEncoder()

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train['species'])


LabelEncoder()

In [ ]:
for idx, label in enumerate(label_encoder.classes_):
    print(f"{label} - {idx}")

Adelie - 0
Gentoo - 1


In [ ]:
y_train['species'] = label_encoder.transform(y_train['species'])
y_test['species'] = label_encoder.transform(y_test['species'])

In [ ]:
x_new_train = one_hot_encoder.transform(x_train['island'].values.reshape(-1, 1)).toarray()
encoded_columns = one_hot_encoder.get_feature_names_out(['island'])
encoded_df = pd.DataFrame(x_new_train, columns=encoded_columns)
x_train = pd.concat([x_train, encoded_df], axis=1)
x_train.drop(columns=['island'], inplace=True)

In [ ]:
x_new_test = one_hot_encoder.transform(x_test['island'].values.reshape(-1, 1)).toarray()
encoded_columns = one_hot_encoder.get_feature_names_out(['island'])
encoded_df = pd.DataFrame(x_new_test, columns=encoded_columns)
x_test = pd.concat([x_test, encoded_df], axis=1)
x_test.drop(columns=['island'], inplace=True)

**Обучение алгоритма**

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train['species'])

LogisticRegression()

In [ ]:
answers_pred = model.predict(x_test)

**Оценка работы**

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, answers_pred).ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)


TN: 39
FP: 1
FN: 0
TP: 29


**Вывод:** Модель допустила всего лишь 1 ошибку(Определила негативный класс как положительный - ложное срабатывание)

In [ ]:
accuracy_score(y_test, answers_pred)

0.9855072463768116

**Вывод:** Точность составляет 99%

In [ ]:
recall_score(y_test, answers_pred)

1.0

In [ ]:
precision_score(y_test, answers_pred)

0.9666666666666667

In [ ]:
f1_score(y_test, answers_pred)

0.9830508474576272